# Fox Study Eyetracking - Television and Cellphone Conditions
By: Elizabeth Beard

Updated: 20241007

This notebook:

- wrangles all participant level ET data to analyze at the subject level and calculate relevant metrics:

    - Total dwell time: Measured in seconds. This represents the total time the participant spent looking at the screen for a particular commercial.
    - Total number of fixations: A count of unique fixations that occurred during the commercial.
    - Average fixation duration: Measured in seconds. This is the average time spent on each fixation.
    - Time spent within AOI: Measured in seconds. This represents the total time spent looking inside the commercial AOI. Same as dwell time.
    - Time spent outside AOI: Measured in seconds. This represents the total time spent looking outside the commercial AOI.
    - Total Ad Time: which represents the total time the ad was displayed in seconds. This metric corresponds to the duration of each commercial based on the difference between the maximum and minimum timestamps for each section (commercial).
- creates average ET variables by condition, by advertisement


In [1]:
import numpy as np
import pandas as pd
import os
import csv
import glob

In [2]:
# paths and files for subj looping
t7sheild = os.path.join('/Volumes/T7 Shield/fox')
subj_folders = glob.glob(os.path.join(t7sheild, 'et', 'sub-[0-9][0-9][0-9]_*'))
subj_folders = [folder for folder in subj_folders if 'control' not in folder]
subj_folders.sort()
print(len(subj_folders))
subj_folders

138


['/Volumes/T7 Shield/fox/et/sub-019_tv-lego-42b0cb7d',
 '/Volumes/T7 Shield/fox/et/sub-021_tv-pickers-97606087',
 '/Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780',
 '/Volumes/T7 Shield/fox/et/sub-023_tv-continental-594a5c2b',
 '/Volumes/T7 Shield/fox/et/sub-024_tv-fox-96dfc1c4',
 '/Volumes/T7 Shield/fox/et/sub-025_tv-nfl-a6e3e656',
 '/Volumes/T7 Shield/fox/et/sub-027_tv-lego-2e4ef2bf',
 '/Volumes/T7 Shield/fox/et/sub-028_tv-bigmood-76d32333',
 '/Volumes/T7 Shield/fox/et/sub-029_tv-pickers-bf2ffb67',
 '/Volumes/T7 Shield/fox/et/sub-030_tv-cnn-b8d33fbe',
 '/Volumes/T7 Shield/fox/et/sub-031_tv-continental-8c85bd3f',
 '/Volumes/T7 Shield/fox/et/sub-032_tv-fox-b5beca97',
 '/Volumes/T7 Shield/fox/et/sub-033_tv-nfl-3d751d85',
 '/Volumes/T7 Shield/fox/et/sub-035_tv-lego-27d35985',
 '/Volumes/T7 Shield/fox/et/sub-036_tv-bigmood-2a7adbc3',
 '/Volumes/T7 Shield/fox/et/sub-037_tv-pickers-f8e0659f',
 '/Volumes/T7 Shield/fox/et/sub-038_tv-cnn-f65c3e71',
 '/Volumes/T7 Shield/fox/et/sub-039_tv-cont

# Create ET Metrics

In [15]:
tv = {'2':1,
           '3':2,
           '4':3,
           '5':4,
           '7':5,
           '8':6,
           '9':7,
           '10':8
}

phone_a = {1: 'yt',
           2: 'yt',
           3: 'yt',
           4: 'yt',
           5: 'fb',
           6: 'fb',
           7: 'fb',
           8: 'fb'
}

phone_b = {5: 'yt',
           6: 'yt',
           7: 'yt',
           8: 'yt',
           1: 'fb',
           2: 'fb',
           3: 'fb',
           4: 'fb'
}

In [42]:
all_metrics = {
    'Subject ID': None,
    'Device': None,
    'Condition': None,
    'Ad ID': None,
    'Total Viewing Time (seconds)': None,
    'Dwell Time (seconds)': None,
    'Number of Fixations': None,
    'Average Fixation Duration (seconds)': None,
    'Total Gaze Duration (seconds)': None,
    'Percentage of Time in AOI (%)': None
}

all_metrics_df = pd.DataFrame([all_metrics])

for subj in subj_folders:
    subj_id = subj.split('/')[-1].split('_')[0]
    condition = subj.split('/')[-1].split('_')[1]

    # print(subj_id, condition)

    if 'tv' in condition:
        device = 'tv'
        ad_condition = condition.split('-')[1]

        if ad_condition == "fox":
            ad_condition = "foxnews"
    elif 'phone' in condition:
        device = 'phone'
        ad_condition = None

    ad_csvs = glob.glob(os.path.join(subj, 'ads', '[0-9]*', 'updated_gaze_data.csv'))
    ad_csvs.sort()

    if len(ad_csvs) == 0:
        continue

    else:
        for ad in ad_csvs:
            ad_id = ad.split('/')[-2]
            ad_id = ad_id.split('_')[0]

            if 'tv' in condition:
                ad_id = tv[ad_id]

            if 'phone-a' in condition:
                ad_condition = phone_a[int(ad_id)]
            elif 'phone-b' in condition:
                ad_condition = phone_b[int(ad_id)]


            # print(ad_id, ad_condition)

            ad_df = pd.read_csv(ad)

            # Convert the timestamp to a more usable format (from nanoseconds to seconds)
            ad_df['timestamp [s]'] = ad_df['timestamp [ns]'] / 1e9

            # Filter data to include only rows where the gaze was within the AOI
            gaze_aoi = ad_df[ad_df['within_aoi'] == True]

            # 1. Dwell Time: Time spent in AOI
            dwell_time = gaze_aoi['timestamp [s]'].max() - gaze_aoi['timestamp [s]'].min()

            # 2. Number of Fixations: Count unique fixation IDs within AOI
            num_fixations = gaze_aoi['fixation id'].nunique()

            # 3. Average Fixation Duration: Calculate duration for each fixation and then average it
            fixation_durations = gaze_aoi.groupby('fixation id')['timestamp [s]'].apply(lambda x: x.max() - x.min())
            average_fixation_duration = fixation_durations.mean()

            # 1. Total Gaze Duration: The sum of all the time spent in fixations within the AOI
            total_gaze_duration = fixation_durations.sum()

            # 2. Percentage of Time in AOI: The proportion of total viewing time spent in the AOI
            # Total viewing time is the time from the first to the last timestamp in the dataset
            total_viewing_time = ad_df['timestamp [s]'].max() - ad_df['timestamp [s]'].min()
            percentage_time_in_aoi = (total_gaze_duration / total_viewing_time) * 100


            # Combine all the metrics into a single dictionary with the recording id
            ad_metrics = {
                'Subject ID': subj_id,
                'Device': device,
                'Condition': ad_condition,
                'Ad ID': int(ad_id),
                'Total Viewing Time (seconds)': total_viewing_time,
                'Dwell Time (seconds)': dwell_time,
                'Number of Fixations': num_fixations,
                'Average Fixation Duration (seconds)': average_fixation_duration,
                'Total Gaze Duration (seconds)': total_gaze_duration,
                'Percentage of Time in AOI (%)': percentage_time_in_aoi
            }

            # Convert the dictionary to a DataFrame
            all_metrics_df = pd.concat([all_metrics_df, pd.DataFrame([ad_metrics])])

all_metrics_df = all_metrics_df.dropna(how='all')

/var/folders/4z/r7mkx_mj2d73zrgl4w0848ww0000gq/T/ipykernel_66569/1908480839.py:96: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_metrics_df = pd.concat([all_metrics_df, pd.DataFrame([ad_metrics])])


In [43]:
all_metrics_df.to_csv(os.path.join(t7sheild, 'subj_et_metrics.csv'), index=False)

# Average Across Condition and Ad ID

In [39]:
ad_ids = {1:'BURGERKING',
2: 'CHEVROLET',
3: 'MOUNJARO',
4: 'PARAMOUNT',
5: 'RAKUTEN', 
6: 'REDBULL',
7: 'SAMSUNG',
8: 'TRACFONE'
}

In [44]:
# Convert relevant columns to numeric
numeric_columns = [
    'Total Viewing Time (seconds)',
    'Dwell Time (seconds)',
    'Number of Fixations',
    'Average Fixation Duration (seconds)',
    'Total Gaze Duration (seconds)',
    'Percentage of Time in AOI (%)'
]

all_metrics_df[numeric_columns] = all_metrics_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [45]:
metrics_by_condition = all_metrics_df.groupby(['Condition', 'Ad ID'])[numeric_columns].mean().reset_index()

In [46]:
metrics_by_condition['Brand'] = metrics_by_condition['Ad ID'].map(ad_ids)

In [48]:
metrics_by_condition.to_csv(os.path.join(t7sheild, 'et_metrics_by_condition-20241007.csv'), index=False)